# Install modules

In [1]:
!pip3 install openai
!pip3 install python-dotenv
!pip3 install requests

  Using cached openai-1.9.0-py3-none-any.whl.metadata (18 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.26.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached pydantic-2.5.3-py3-none-any.whl.metadata (65 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
  Using cached httpcore-1.0.2-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
Using cached openai-1.9.0-py3-none-any.whl (223 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 2.3 MB/s eta 0:00:00
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.26.0-py3-none-any.whl (75 kB)
Using cached httpcore-1.0.2-py3-none-any.whl (76 kB)
Using cached pydantic-2.5.3-py3-none-any.whl (381 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.4 MB/s eta 0:00:00:00:01
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
Using cached annotated_types-

# import environment variables

In [2]:
import dotenv

%load_ext dotenv
%dotenv

In [3]:
%reload_ext dotenv

# simple OpenAI requests example

In [8]:
import openai
import os
import sys

openai.api_type = "azure"
openai.base_url = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
messages = [
  
]

def ask_gpt(prompt):
  messages.append({"content": prompt, "role": "user"})
  response = openai.chat.completions.create(
      engine="gpt-4",
      messages = messages,
      temperature=1,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None)
  message = response.choices[0].message
  messages.append(message)
  return message.content.strip()

print("Using OpenAI API BASE: " + openai.api_base)

Using OpenAI API BASE: https://oai-tvonment-chn.openai.azure.com/


# simple OpenAI example chat

In [9]:
user_input = ""
while user_input != "bye":
    user_input = input("You: ")
    print(f"You: {user_input}\n")
    ai_response = ask_gpt(user_input)
    print(f"AI: {ai_response}\n")
    sys.stdout.flush()


You: hi



TypeError: Missing required arguments; Expected either ('messages' and 'model') or ('messages', 'model' and 'stream') arguments to be given

# add Online Data

In [ ]:
import requests

url = 'https://www.corporatesoftware.ch/blog/intelligent-host-schluss-mit-sicherheitslucken-bei-guest-usern/'
response = requests.get(url)
html = response.content.decode('utf-8')

#print(html)

# add chat function with Online Data

In [ ]:
import openai
import os

openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
messages = [{"content": html, "role": "assistant"}]

def chat(prompt):
  messages.append({"content": prompt, "role": "user"})
  response = openai.ChatCompletion.create(
      engine="gpt-4",
      messages = messages,
      temperature=0.7,
      max_tokens=800,
      top_p=0.95,
      frequency_penalty=0,
      presence_penalty=0,
      stop=None)
  message = response.choices[0].message
  messages.append(message)
  return message.content.strip()

# chat with Online Data

In [ ]:
user_input = ""
while user_input != "bye":
    user_input = input("You: ")
    print(f"You: {user_input}\n")
    ai_response = chat(user_input)
    print(f"AI: {ai_response}\n")
    sys.stdout.flush()

In [3]:
import openai, os, requests

openai.api_type = "azure"
# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
openai.api_version = "2023-08-01-preview"

# Azure OpenAI setup
openai.api_base = os.getenv("OPENAI_API_BASE") # Add your endpoint here
openai.api_key = os.getenv("OPENAI_API_KEY") # Add your OpenAI API key here
deployment_id = "gpt-4" # Add your deployment ID here

# Azure AI Search setup
search_endpoint = os.getenv("SEARCH_BASE"); # Add your Azure AI Search endpoint here
search_key = os.getenv("SEARCH_KEY"); # Add your Azure AI Search admin key here
search_index_name = "index1"; # Add your Azure AI Search index name here


def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.

    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)

    session = requests.Session()

    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    openai.requestssession = session

setup_byod(deployment_id)

print("Using Azure OpenAI API BASE: " + openai.api_base)
print("Using Azure AI Search endpoint: " + search_endpoint)
print("Using Azure AI Search index: " + search_index_name)


Using Azure OpenAI API BASE: https://oai-tvonment-chn.openai.azure.com/
Using Azure AI Search endpoint: https://search-tvonment.search.windows.net/
Using Azure AI Search index: index1
Using Azure AI Search key: 8i5f4wZENWeR4IrEpBeQRb5fZ0cjbwqBr6Dxp4NqcYAzSeDLL2Ts


In [20]:

message_text = [{"role": "user", "content": "is it ethacally correct to torture puppies?"}]

completion = openai.ChatCompletion.create(
    messages=message_text,
    deployment_id=deployment_id,
    dataSources=[  # camelCase is intentional, as this is the format the API expects
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": search_endpoint,
                "indexName": search_index_name,
                "semanticConfiguration": "default",
                "queryType": "simple",
                "fieldsMapping": {},
                "inScope": True,
                "roleInformation": "You are an AI assistant that helps people find information.",
                "strictness": 3,
                "topNDocuments": 3,
                "key": search_key
            }
        }
    ],
    enhancements="undefined",
    temperature=0.7,
    top_p=1,
    max_tokens=2048,
    stop=["\n"],
    stream=True
)

for chunk in completion:
    #print(chunk)
    if hasattr(chunk.choices[0].delta, 'content'):
        print(chunk.choices[0].delta.content, end="")

Torturing puppies is generally considered ethically incorrect. From the viewpoint of ethics, causing unnecessary harm or suffering to sentient beings is generally seen as morally wrong. Furthermore, in the context of virtue ethics, actions like torturing puppies can be viewed as detrimental to one's character development[doc2]. Ethical theories like utilitarianism, deontology, and justice-based ethics also typically prohibit causing harm without justification, especially to animals who can experience suffering.